In [3]:
# pull in dependencies needed from sportsreference api
from sportsreference.mlb.teams import Teams
from sportsreference.mlb.teams import Roster
from sportsreference.mlb.roster import Player

# pull in all other dependencies needed
import itertools
import pandas as pd


In [ ]:
# to create a list of player ids:
# set up an empty array that we will append to
player_list = []

# 1) create a loop to go through years
for year in range(2019,2020):

# 2) create a loop to go through teams and pull out the player names and their ids
    for team in Teams(year):
        player_data = Roster(team.abbreviation, year = year, slim=True).players
        player_list.append(player_data)

In [5]:
# convert the dictionay of arrays to an array of arrays using list comprehension
info = [list(x.keys()) for x in player_list]

In [6]:
# flatten the list using itertools
merged = list(itertools.chain(*info))

In [7]:
# create a pandas data frame that will show all of the player ids
df = pd.DataFrame(merged)
df

,0
0,chiriro01
1,gourryu01
2,altuvjo01
3,correca01
4,bregmal01
...,...
1562,garrere01
1563,greensh02
1564,reiniza01
1565,rossty01


In [8]:
# use .unique() on the df to clear out any duplicate players
unique_player_id = df[0].unique()
unique_player_id

array(['chiriro01', 'gourryu01', 'altuvjo01', ..., 'garrere01',
       'reiniza01', 'rossty01'], dtype=object)

In [10]:
# to create a list of data frames for each player's stats:
# set up an empty array that we will append to
player_df_list = []

# create a loop that will find each players' id and create a data frame of relevant statistics
for index, player_id in enumerate(unique_player_id):
    player_df = Player(player_id).dataframe[["player_id", "name", "at_bats", "runs", "hits", "runs_batted_in", "bases_on_balls", "batting_average", "doubles", "triples", "home_runs", "stolen_bases"]]
    player_df_list.append(player_df)
    print(index, len(unique_player_id))

0 1410
1 1410
2 1410
3 1410
4 1410
5 1410
6 1410
7 1410
8 1410
9 1410
10 1410
11 1410
12 1410
13 1410
14 1410
15 1410
16 1410
17 1410
18 1410
19 1410
20 1410
21 1410
22 1410
23 1410
24 1410
25 1410
26 1410
27 1410
28 1410
29 1410
30 1410
31 1410
32 1410
33 1410
34 1410
35 1410
36 1410
37 1410
38 1410
39 1410
40 1410
41 1410
42 1410
43 1410
44 1410
45 1410
46 1410
47 1410
48 1410
49 1410
50 1410
51 1410
52 1410
53 1410
54 1410
55 1410
56 1410
57 1410
58 1410
59 1410
60 1410
61 1410
62 1410
63 1410
64 1410
65 1410
66 1410
67 1410
68 1410
69 1410
70 1410
71 1410
72 1410
73 1410
74 1410
75 1410
76 1410
77 1410
78 1410
79 1410
80 1410
81 1410
82 1410
83 1410
84 1410
85 1410
86 1410
87 1410
88 1410
89 1410
90 1410
91 1410
92 1410
93 1410
94 1410
95 1410
96 1410
97 1410
98 1410
99 1410
100 1410
101 1410
102 1410
103 1410
104 1410
105 1410
106 1410
107 1410
108 1410
109 1410
110 1410
111 1410
112 1410
113 1410
114 1410
115 1410
116 1410
117 1410
118 1410
119 1410
120 1410
121 1410
122 1410
123

In [23]:
# Use .loc function in pandas to just obtain the career stats for every player
career_df_list = []
for df in player_df_list:
    career_df_list.append(df.loc["Career"])
career_df_list

[        player_id               name  at_bats  hits  batting_average  doubles  \
 Career  chiriro01  Robinson Chirinos     1832   423            0.231      100   
 
         triples  home_runs  runs_batted_in  
 Career        4         86             269  ,
         player_id          name  at_bats  hits  batting_average  doubles  \
 Career  gourryu01  Yuli Gurriel     1971   565            0.287      135   
 
         triples  home_runs  runs_batted_in  
 Career        5         71             301  ,
         player_id         name  at_bats  hits  batting_average  doubles  \
 Career  altuvjo01  José Altuve     5177  1610            0.311      308   
 
         triples  home_runs  runs_batted_in  
 Career       28        133             556  ,
         player_id           name  at_bats  hits  batting_average  doubles  \
 Career  correca01  Carlos Correa     2269   626            0.276      128   
 
         triples  home_runs  runs_batted_in  
 Career        7        107             3

In [25]:
# use concat to re-write the list of data frames as a single data frame
clean_df = pd.concat(career_df_list)

# and drop the duplicates of names using either 'player_id' or 'name'
no_duplicates_df = clean_df.drop_duplicates(subset=['player_id'])
no_duplicates_df

,player_id,name,at_bats,hits,batting_average,doubles,triples,home_runs,runs_batted_in
Career,chiriro01,Robinson Chirinos,1832,423.0,0.231,100,4,86.0,269
Career,gourryu01,Yuli Gurriel,1971,565.0,0.287,135,5,71.0,301
Career,altuvjo01,José Altuve,5177,1610.0,0.311,308,28,133.0,556
Career,correca01,Carlos Correa,2269,626.0,0.276,128,7,107.0,397
Career,bregmal01,Alex Bregman,2058,582.0,0.283,152,12,105.0,342
...,...,...,...,...,...,...,...,...,...
Career,schrejo01,John Schreiber,None,35.0,None,None,None,5.0,None
Career,hardybl01,Blaine Hardy,1,0.0,0,0,0,0.0,0
Career,garrere01,Reed Garrett,None,24.0,None,None,None,3.0,None
Career,reiniza01,Zac Reininger,0,0.0,None,0,0,0.0,0


In [9]:
# read in the hall of fame csv
hall_of_fame_df = pd.read_csv('Resources/Hall_of_Fame_List.csv')
hall_of_fame_df 

,Year,name,Primary position,Primary team,Career,Induction method,Vote %
0,1936.0,Ty Cobb,CF,Detroit Tigers,1905–1928,BBWAA,98.23%
1,1936.0,Walter Johnson,P,Washington Senators,1907–1927,BBWAA,83.63%
2,1936.0,Christy Mathewson,P,New York Giants,1900–1916,BBWAA,90.71%
3,1936.0,Babe Ruth,RF,New York Yankees,1914–1935,BBWAA,95.13%
4,1936.0,Honus Wagner,SS,Pittsburgh Pirates,1897–1917,BBWAA,95.13%
...,...,...,...,...,...,...,...
452,2019.0,Lee Smith,P,Chicago Cubs,1980–1997,VC,100%
454,20206.0,Derek Jeter,SS,New York Yankees,1995–2014,BBWAA,99.75%
455,20206.0,Larry Walker,RF,Colorado Rockies,1989–2005,BBWAA,76.57%
456,20206.0,Marvin Miller,EXEC,—,1966–1982,VC,75%


In [11]:
# clean the csv file to show only the names of the hall of famers
hall_of_fame_df = hall_of_fame_df.dropna()
hof_df = hall_of_fame_df.rename(columns={"Name":"name"})
hof_cleaned = hof_df[["name"]]
hof_cleaned

,name
0,Ty Cobb
1,Walter Johnson
2,Christy Mathewson
3,Babe Ruth
4,Honus Wagner
...,...
452,Lee Smith
454,Derek Jeter
455,Larry Walker
456,Marvin Miller


In [ ]:
# join the no_duplicates_df with hof_cleaned to show career stats and hall of fame status
merge_df = no_duplicates.merge(hall_of_fame_df, on='name', how='inner')